In [16]:
import sqlite3
import pandas as pd

df=pd.read_csv("SuperMarket Analysis.csv")
df['Date']=pd.to_datetime(df['Date'], format='%m/%d/%Y')
conn=sqlite3.connect("sales.db")

df.to_sql("sales",conn,if_exists="replace",index=False)

pd.read_sql("SELECT* FROM sales ;",conn)

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Sales,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,Alex,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,2019-01-05 00:00:00,1:08:00 PM,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,Giza,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,2019-03-08 00:00:00,10:29:00 AM,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,Alex,Yangon,Normal,Female,Home and lifestyle,46.33,7,16.2155,340.5255,2019-03-03 00:00:00,1:23:00 PM,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,Alex,Yangon,Member,Female,Health and beauty,58.22,8,23.2880,489.0480,2019-01-27 00:00:00,8:33:00 PM,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,Alex,Yangon,Member,Female,Sports and travel,86.31,7,30.2085,634.3785,2019-02-08 00:00:00,10:37:00 AM,Ewallet,604.17,4.761905,30.2085,5.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,233-67-5758,Giza,Naypyitaw,Normal,Male,Health and beauty,40.35,1,2.0175,42.3675,2019-01-29 00:00:00,1:46:00 PM,Ewallet,40.35,4.761905,2.0175,6.2
996,303-96-2227,Cairo,Mandalay,Normal,Female,Home and lifestyle,97.38,10,48.6900,1022.4900,2019-03-02 00:00:00,5:16:00 PM,Ewallet,973.80,4.761905,48.6900,4.4
997,727-02-1313,Alex,Yangon,Member,Male,Food and beverages,31.84,1,1.5920,33.4320,2019-02-09 00:00:00,1:22:00 PM,Cash,31.84,4.761905,1.5920,7.7
998,347-56-2442,Alex,Yangon,Normal,Male,Home and lifestyle,65.82,1,3.2910,69.1110,2019-02-22 00:00:00,3:33:00 PM,Cash,65.82,4.761905,3.2910,4.1


Revenue by Branch

In [7]:
query="""
SELECT Branch, SUM(Sales) as total_revenue
FROM sales
GROUP BY Branch
ORDER BY total_revenue DESC;
"""
df_branch_revenue=pd.read_sql(query,conn)
df_branch_revenue



,Branch,total_revenue
0,Giza,110568.7065
1,Alex,106200.3705
2,Cairo,106197.6720


Monthly sales trend

In [14]:
query="""
SELECT strftime('%Y-%m',Date) as month,SUM(Sales) as total_revenue
FROM sales
GROUP BY month
ORDER BY total_revenue DESC;
"""
df_monthly_sales=pd.read_sql(query,conn)
df_monthly_sales

,month,total_revenue
0,2019-01,116291.868
1,2019-03,109455.507
2,2019-02,97219.374


Top Product Lines

In [22]:
query="""
SELECT "Product line",SUM(Sales) as total_revenue
FROM sales
GROUP BY "Product line"
ORDER BY total_revenue DESC;
"""
df_product_revenue=pd.read_sql(query,conn)
df_product_revenue

,Product line,total_revenue
0,Food and beverages,56144.8440
1,Sports and travel,55122.8265
2,Electronic accessories,54337.5315
3,Fashion accessories,54305.8950
4,Home and lifestyle,53861.9130
5,Health and beauty,49193.7390


Average ratings per product line

In [23]:
query="""
SELECT "Product line",AVG(Rating) as avg_rating
FROM sales
GROUP BY "Product line"
ORDER BY avg_rating DESC;
"""
df_product_rating=pd.read_sql(query,conn)
df_product_rating

,Product line,avg_rating
0,Food and beverages,7.113218
1,Fashion accessories,7.029213
2,Health and beauty,7.003289
3,Electronic accessories,6.924706
4,Sports and travel,6.916265
5,Home and lifestyle,6.837500


Revenue by customer type

In [24]:
query="""
SELECT "Customer type",SUM(Sales) as total_revenue
FROM sales
GROUP BY "Customer type"
ORDER BY total_revenue DESC;
"""
df_customer_revenue=pd.read_sql(query,conn)
df_customer_revenue

,Customer type,total_revenue
0,Member,189694.764
1,Normal,133271.985


Cities by sales


In [25]:
query="""
SELECT City,SUM(Sales) as total_revenue
FROM sales
GROUP BY City
ORDER BY total_revenue DESC;
"""
df_city_revenue=pd.read_sql(query,conn)
df_city_revenue

,City,total_revenue
0,Naypyitaw,110568.7065
1,Yangon,106200.3705
2,Mandalay,106197.6720


Mostly used payment methods

In [26]:
query="""
SELECT Payment,COUNT(*) as payment_count
FROM sales
GROUP BY Payment
ORDER BY payment_count DESC;
"""
df_payment_count=pd.read_sql(query,conn)
df_payment_count

,Payment,payment_count
0,Ewallet,345
1,Cash,344
2,Credit card,311


Highest value transactions

In [29]:
query="""
SELECT* FROM sales
WHERE Sales>(SELECT AVG(Sales) FROM sales
)
ORDER BY Sales DESC;

"""
df_highest_value=pd.read_sql(query,conn)
df_highest_value


,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Sales,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,860-79-0874,Giza,Naypyitaw,Member,Female,Fashion accessories,99.30,10,49.6500,1042.6500,2019-02-15 00:00:00,2:53:00 PM,Credit card,993.00,4.761905,49.6500,6.6
1,687-47-8271,Alex,Yangon,Normal,Male,Fashion accessories,98.98,10,49.4900,1039.2900,2019-02-08 00:00:00,4:20:00 PM,Credit card,989.80,4.761905,49.4900,8.7
2,283-26-5248,Giza,Naypyitaw,Member,Female,Food and beverages,98.52,10,49.2600,1034.4600,2019-01-30 00:00:00,8:23:00 PM,Ewallet,985.20,4.761905,49.2600,4.5
3,751-41-9720,Giza,Naypyitaw,Normal,Male,Home and lifestyle,97.50,10,48.7500,1023.7500,2019-01-12 00:00:00,4:18:00 PM,Ewallet,975.00,4.761905,48.7500,8.0
4,303-96-2227,Cairo,Mandalay,Normal,Female,Home and lifestyle,97.38,10,48.6900,1022.4900,2019-03-02 00:00:00,5:16:00 PM,Ewallet,973.80,4.761905,48.6900,4.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399,443-82-0585,Alex,Yangon,Member,Female,Health and beauty,77.68,4,15.5360,326.2560,2019-02-01 00:00:00,7:54:00 PM,Cash,310.72,4.761905,15.5360,8.4
400,220-68-6701,Alex,Yangon,Normal,Female,Home and lifestyle,77.47,4,15.4940,325.3740,2019-03-17 00:00:00,4:36:00 PM,Cash,309.88,4.761905,15.4940,4.2
401,563-91-7120,Alex,Yangon,Normal,Female,Fashion accessories,61.77,5,15.4425,324.2925,2019-03-08 00:00:00,1:21:00 PM,Cash,308.85,4.761905,15.4425,6.7
402,434-83-9547,Giza,Naypyitaw,Member,Male,Food and beverages,38.47,8,15.3880,323.1480,2019-01-23 00:00:00,11:51:00 AM,Cash,307.76,4.761905,15.3880,7.7


Average basket size per branch

In [30]:
query="""
SELECT Branch,AVG(Sales) as avg_basket_size
FROM sales
GROUP BY Branch;
"""
df_avg_basket_size=pd.read_sql(query,conn)
df_avg_basket_size

,Branch,avg_basket_size
0,Alex,312.354031
1,Cairo,319.872506
2,Giza,337.099715


Day with the highest sales

In [34]:
query="""
SELECT CASE strftime('%w',Date)
WHEN '0' THEN 'Sunday'
WHEN '1' THEN 'Monday'
WHEN '2' THEN 'Tuesday'
WHEN '3' THEN 'Wednesday'
WHEN '4' THEN 'Thursday'
WHEN '5' THEN 'Friday'
WHEN '6' THEN 'Saturday'
END AS day_of_week,
SUM(Sales) as total_revenue
FROM sales
GROUP BY day_of_week
ORDER BY total_revenue DESC;
"""
df_day_sales=pd.read_sql(query,conn)
df_day_sales

,day_of_week,total_revenue
0,Saturday,56120.8095
1,Tuesday,51482.2455
2,Thursday,45349.2480
3,Sunday,44457.8925
4,Friday,43926.3405
5,Wednesday,43731.1350
6,Monday,37899.0780
